### Import Modules

In [55]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [56]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

### Input Data

In [57]:
Recyling_Products = [12000, 8000, 19000, 21000, 25000, 5000]
Construction_Costs = [12000, 19500, 32000]
Handling_Charges = [6, 2, 8]
Processing_Capacity = [0, 38000, 18100]

Cik = [
    [1, 9, 9, 4, 9],
    [8, 4, 2, 7, 4],
    [8, 3, 6, 6, 6],
    [1, 2, 5, 4, 4],
    [9, 6, 4, 1, 8],
    [6, 3, 2, 2, 4]
]

Ckl = [
    [8, 9, 7, 9],
    [11, 8, 4, 10],
    [9, 3, 5, 4],
    [5, 4, 8, 6],
    [7, 6, 3, 8]
]

Cki = [
    [6, 5, 4, 6, 7, 2],
    [4, 2, 6, 3, 7, 2],
    [2, 3,1, 5, 8, 2], 
    [7, 4, 5, 7, 5, 9]
]

### Creating Model and Declaring Variables

In [58]:

model = gp.Model("closed_loop_supply_chain")


Customer_Centres = model.addVars(5, vtype = gp.GRB.BINARY, name = 'Customer_Centres')
Split_Testing_Centres = model.addVars(6, vtype=gp.GRB.BINARY, name="Split_Testing_Centres")
Remanufacturing_Plants = model.addVars(4, vtype=GRB.BINARY, name="Remanufacturing_Plants")


quantity_ij = model.addVars(6, 5, name="quantity_ij", lb=0)
quantity_jk = model.addVars(5, 4, name="quantity_jk", lb=0)
quantity_kl = model.addVars(4, 6, name="quantity_kl", lb=0)

### Objective Function

In [59]:
Construction_Cost_Centres = gp.quicksum(Construction_Costs[0] * Customer_Centres[i] for i in range (5))
Construction_Cost_Split = gp.quicksum(Construction_Costs[1] * Split_Testing_Centres[j] for j in range(6))
Construction_Cost_Remanufacturing = gp.quicksum(Construction_Costs[2] * Remanufacturing_Plants[l] for l in range(4))


Handling_Cost_1 = gp.quicksum(Handling_Charges[0] * quantity_ij[i, j] for i in range(6) for j in range(5))
Handling_Cost_2 = gp.quicksum(Handling_Charges[1] * quantity_jk[j, k] for j in range(5) for k in range(4))
Handling_Cost_3 = gp.quicksum(Handling_Charges[2] * quantity_kl[k, l] for k in range(4) for l in range(6))


Processing_Cost_1 = gp.quicksum(Cik[i][j] * quantity_ij[i, j] for i in range(6) for j in range(5))
Processing_Cost_2 = gp.quicksum(Ckl[j][k] * quantity_jk[j, k] for j in range(5) for k in range(4))
Processing_Cost_3 = gp.quicksum(Cki[k][i] * quantity_kl[k, i] for k in range(4) for i in range(6))

model.setObjective(Construction_Cost_Split + Construction_Cost_Remanufacturing + 
               Handling_Cost_1 + Processing_Cost_1 + Processing_Cost_2 + Processing_Cost_3 + Construction_Cost_Centres + 
               Handling_Cost_2 + Handling_Cost_3
               , GRB.MINIMIZE)


### Constraints and Optimizing the Model

In [60]:

for j in range(5):
    model.addConstr(gp.quicksum(quantity_ij[i, j] for i in range(5)) <= Processing_Capacity[1] * Split_Testing_Centres[j])

for k in range(4):
    model.addConstr(gp.quicksum(quantity_jk[j, k] for j in range(4)) <= Processing_Capacity[2] * Remanufacturing_Plants[k])

for i in range(6):
    model.addConstr(gp.quicksum(quantity_ij[i, j] for j in range(3)) == Recyling_Products[i])


for k in range(4):
    model.addConstr(gp.quicksum(quantity_jk[j, k] for j in range(4)) == gp.quicksum(quantity_kl[k, i] for i in range(5)))


for i in range(4):
    model.addConstr(Customer_Centres[i] <=1)

for j in range(5):
    model.addConstr(Split_Testing_Centres[j] <= 1)
    
for k in range(4):
    model.addConstr(Remanufacturing_Plants[k] <= 1)

    
model.addConstr(gp.quicksum(Split_Testing_Centres[j] for j in range(3)) >= 1, "Minimum-One-Split-Testing-Centre")
model.addConstr(gp.quicksum(Remanufacturing_Plants[k] for k in range(4)) >= 1, "Minimum-Remanafacturing-Plant")
model.addConstr(gp.quicksum(Remanufacturing_Plants[i] for i in range(4)) >= 1, "Minimum-Customer_Centres")

model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 35 rows, 89 columns and 128 nonzeros
Model fingerprint: 0x7d788d1a
Variable types: 74 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [5e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]
Found heuristic solution: objective 891500.00000
Presolve removed 26 rows and 71 columns
Presolve time: 0.00s
Presolved: 9 rows, 18 columns, 36 nonzeros
Found heuristic solution: objective 871500.00000
Variable types: 15 continuous, 3 integer (3 binary)

Root relaxation: objective 8.316184e+05, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I